# Creating Pose-To-Pose Animations with a Robot

When building animations from scratch it can be a big help to see the pose on a real robot before adding it to an animation. This notebook walks through capturing the current joint state of a robot (posed using the spot tablet or other 3rd party tool) and adding it as the next frame in an animation

# Connect To Robot

In [ ]:
import logging
import time
from spot_wrapper.wrapper import SpotWrapper

hostname="__ROBOT_IP__"
robot_name="__ROBOT_NAME__"
username="__USERNAME__"
password="__PASSWORD__"
has_arm = True
port = 0
logger = logging.Logger(name="spot_wrapper_logger")

spot_wrapper = SpotWrapper(
    username=username, 
    password=password, 
    hostname=hostname, 
    port=port, 
    robot_name=robot_name, 
    logger=logger, 
    use_take_lease=True,
    get_lease_on_action=True
)

assert(spot_wrapper.is_valid)
spot_wrapper.claim()

## Create an Animation Builder

In [ ]:
from spot_choreo_utils.choreo_creation.choreo_builders.animation_builder import AnimationBuilder

ANIMATION_NAME = "new_animation"
animation_builder = AnimationBuilder()
animation_builder.start_from_empty(name=ANIMATION_NAME)

## Capture and add a pose

Use a tool to pose the robot for its next keyframe. Then run the next cell to capture the current robot joint angles.

You can re-run the cell with different poses to build the animation pose by pose. Update the KEYFRAME_TIME variable so that each pose is added with an appropriate offset so that the robot doesn't move in fast, unexpected manners.

In [ ]:
from spatialmath import (
    SE3
)
from spot_choreo_utils.choreo_creation.choreo_builders.animation_proto_utils import joint_angle_keyframe_to_proto
from spot_choreo_utils.choreo_creation.robot_pose_to_pose_utils import get_pose_in_animation_frame, get_world_t_body

# Timestamp to insert the keyframe 
KEYFRAME_TIME = 0

current_body_frame = get_world_t_body(spot_wrapper._robot_state_client.get_robot_state())
joint_angles = get_pose_in_animation_frame(spot_wrapper, current_body_frame)

keyframe_proto = joint_angle_keyframe_to_proto(joint_angles, start_time=KEYFRAME_TIME)
animation_builder.insert_keyframe_at_time(start_time=KEYFRAME_TIME, animation_keyframe=keyframe_proto)

# Upload and play back the animation
Run the next cell to upload and playback your pose-to-pose sequene on robot.

In [ ]:
from spot_choreo_utils.choreo_creation.choreo_builders.animation_operators import create_single_animation_sequence

build_settings = AnimationBuilder.BuildSettings()
build_settings.hold_final_pose_s = 3

# Automatically build the animation and create a single animation sequence with the same name
animation_proto, seq_proto = create_single_animation_sequence(animation_builder, build_settings)
print(animation_proto)
print(seq_proto)
spot_wrapper.claim()
# Upload to robot
spot_wrapper.upload_animation_proto(animation_proto)
spot_wrapper.upload_choreography(seq_proto)
time.sleep(1)
# Playback sequence
spot_wrapper.execute_choreography_by_name(seq_proto.name, start_slice=0)

# Re-Loading Animations from Disk
Once you have some poses that you like, you can save them to disk for later playback and editing. The next cell will save the animation you've been working on and then re-load it from disk and play it on robot.

In [ ]:
from spot_choreo_utils.serialization.serialization_utils import load_animation, save_animation
from spot_choreo_utils.choreo_creation.choreo_builders.animation_builder import AnimationBuilder
from spot_choreo_utils.choreo_creation.choreo_builders.animation_operators import create_single_animation_sequence

# Save the new animation to disk
build_settings = AnimationBuilder.BuildSettings()
build_settings.apply_unique_name = False

animation_proto = animation_builder.build(build_settings)
save_animation(animation_proto, "new_animation")

# Load the example animation
example_anim = load_animation("new_animation/new_animation.pbtxt")

animation_builder = AnimationBuilder()
animation_builder.start_from_animation(example_anim)

build_settings = AnimationBuilder.BuildSettings()
build_settings.hold_final_pose_s = 3
animation_proto, seq_proto = create_single_animation_sequence(animation_builder, build_settings)
spot_wrapper.claim()
# Upload to robot
spot_wrapper.upload_animation_proto(animation_proto)
spot_wrapper.upload_choreography(seq_proto)
# Playback sequence
spot_wrapper.execute_choreography_by_name(seq_proto.name, start_slice=0)

This notebook has focused on creating some initial poses, adding them to an animation, and saving them for later. By building on this example code it should be clear how you could continue to load, edit, and save animations back to disk for iterative development and programatic generation of new Spot choreography.